<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:


      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-19 11:02:26
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.37 C
-------------------
Today PnL: 73.05 K (0.54%)
Current PnL: -17.93 L (-12.2%)
CY Booked + Current PnL: -9.66 L (-6.57%)
-------------------
Total profit:  3.67 L
Total loss:  -21.61 L
-------------------
Total Booked + Current PnL: 17.32 L (14.42%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.04%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.56 L (56.63%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.00,-20.67,H-SC,11.77,237143.0,44123.0,3913.0,1.71,22.86,1.65,24.89,137.0,11.28,1.74,51.54,X40N,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.80,113392.0,9080.0,7575.0,-0.76,8.70,6.68,15.96,9.0,1.20,0.83,27.07,X5K,ATH,METALS
69,SBIN,760.30,863.00,-11.34,M-LC,3.88,219288.0,17808.0,9407.0,-0.02,8.84,4.29,13.51,60.0,1.89,1.61,22.55,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-13.86,H-LC,6.59,172172.0,15995.0,10106.0,0.17,10.24,5.87,16.71,22.0,1.58,1.27,29.47,X40N,BTT,HEALTHCARE
84,TTKPRESTIG,769.29,770.00,102.99,M-SC,9.84,87501.0,-13276.0,13370.0,3.71,-13.17,15.28,0.09,245.0,-0.99,0.64,14.96,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJOOENG,102.50,143.33,-31.84,H-SC,7.57,103400.0,900.0,39933.0,1.74,0.88,38.62,39.83,114.0,0.02,0.76,2.38,AR,ATH,MISC
59,PGHH,13388.00,18062.58,-31.08,H-MC,6.11,199335.0,-1485.0,71601.0,-0.67,-0.74,35.92,34.92,80.0,-0.02,1.47,3.90,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101310.0,806.0,29836.0,1.50,0.80,29.45,30.49,79.0,0.03,0.75,33.47,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.77,H-LC,2.98,147675.0,-2257.0,16850.0,-0.15,-1.51,11.41,9.74,4.0,-0.13,1.09,4.64,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.78,H-LC,3.11,214336.0,9836.0,31572.0,-0.04,4.81,14.73,20.25,10.0,0.31,1.58,13.96,X40N,BTT,PHARMA
58,NESTLEIND,1134.30,1377.0,-13.31,H-LC,3.49,266830.0,1404.0,55394.0,-0.33,0.53,20.76,21.40,11.0,0.03,1.96,7.66,XY25,NTT,FMCG
54,LTIM,5564.16,7230.2,-8.38,H-LC,3.77,182034.0,-18276.0,78256.0,-0.65,-9.12,42.99,29.94,16.0,-0.23,1.34,25.76,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-10.32,H-LC,4.19,174896.0,-26490.0,114102.0,-0.60,-13.15,65.24,43.51,27.0,-0.23,1.29,21.75,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,LAOPALA,369.40,464.0,121.82,H-SC,2.13,70530.0,-30316.0,56142.0,6.14,-30.06,79.60,25.61,142.0,-0.54,0.52,30.76,AR,NTT,CERAMICS
74,SONACOMS,578.05,1006.0,-34.58,M-SC,13.67,81384.0,-19775.0,94666.0,4.52,-19.55,116.32,74.03,202.0,-0.21,0.60,14.76,AR,BTT,AUTO
84,TTKPRESTIG,769.29,770.0,102.99,M-SC,9.84,87501.0,-13276.0,13370.0,3.71,-13.17,15.28,0.09,245.0,-0.99,0.64,14.96,OX40N,NTT,DURABLES
80,TATAMOTORS,761.77,1065.0,-19.53,M-LC,5.22,254282.0,-22241.0,132303.0,3.62,-8.04,52.03,39.81,54.0,-0.17,1.87,21.87,XY24,NTT,AUTO
85,UJJIVANSFB,52.77,60.0,84.36,H-SC,20.16,116397.0,-26082.0,45604.0,3.46,-18.31,39.18,13.70,163.0,-0.57,0.86,37.38,OX40N,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.0,-322.08,H-SC,16.00,94380.0,-38906.0,123496.0,-1.94,-29.19,130.85,63.47,139.0,-0.32,0.69,58.20,XR,NTT,CHEMICALS
77,SYMPHONY,1306.42,1306.0,-40.95,M-SC,13.87,125668.0,-45473.0,45416.0,-1.87,-26.57,36.14,-0.03,196.0,-1.00,0.92,1.59,OX40N,NTT,DURABLES
9,BAJAJFINSV,1994.80,2758.0,-10.13,H-LC,7.11,96554.0,-1191.0,38583.0,-1.15,-1.22,39.96,38.26,35.0,-0.03,0.71,27.10,X40,BTT,FINANCE
14,BERGEPAINT,561.33,680.0,-17.69,H-MC,9.25,221150.0,-6189.0,54248.0,-0.96,-2.72,24.53,21.14,106.0,-0.11,1.63,24.65,XY24,NTT,PAINTS
92,WHIRLPOOL,1219.98,2270.0,-42.81,M-SC,4.76,96622.0,5124.0,73626.0,-0.89,5.60,76.20,86.07,223.0,0.07,0.71,39.77,XR,NTT,DURABLES


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,102.99,M-SC,9.84,87501.0,-13276.0,13370.0,3.71,-13.17,15.28,0.09,245.0,-0.99,0.64,14.96,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,120.98,M-SC,2.25,144919.0,-29640.0,29679.0,2.51,-16.98,20.48,0.02,221.0,-1.00,1.07,25.67,OX40N,NTT,PAINTS
20,CERA,7989.07,9475.0,-17.42,H-SC,3.03,117720.0,-26083.0,52833.0,0.63,-18.14,44.88,18.60,149.0,-0.49,0.87,29.91,OX40N,NTT,CERAMICS
85,UJJIVANSFB,52.77,60.0,84.36,H-SC,20.16,116397.0,-26082.0,45604.0,3.46,-18.31,39.18,13.70,163.0,-0.57,0.86,37.38,OX40N,NTT,BANKS
50,KANSAINER,299.63,340.0,-68.52,H-SC,3.17,217422.0,-52245.0,88578.0,1.99,-19.37,40.74,13.47,138.0,-0.59,1.60,9.97,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-29.01,H-LC,16.19,235209.0,-60725.0,154626.0,0.14,-20.52,65.74,31.73,1.0,-0.39,1.73,0.42,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.03,H-LC,11.59,258064.0,-3537.0,149161.0,0.41,-1.35,57.80,55.67,3.0,-0.02,1.90,4.67,X40,BTT,IT
47,ITC,411.90,452.00,-38.77,H-LC,2.98,147675.0,-2257.0,16850.0,-0.15,-1.51,11.41,9.74,4.0,-0.13,1.09,4.64,X40,NTT,FMCG
90,VBL,492.64,671.64,-10.56,H-LC,9.78,271556.0,6516.0,89776.0,0.99,2.46,33.06,36.33,5.0,0.07,2.00,16.09,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.91,H-LC,11.27,197379.0,-19140.0,112052.0,0.47,-8.84,56.77,42.91,7.0,-0.17,1.45,3.37,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.0,-20.67,H-SC,11.77,237143.0,44123.0,3913.0,1.71,22.86,1.65,24.89,137.0,11.28,1.74,51.54,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.0,11.34,H-MC,13.94,158966.0,32766.0,13417.0,2.02,25.96,8.44,36.59,108.0,2.44,1.17,41.73,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-9.31,H-LC,7.16,226454.0,41973.0,28397.0,1.39,22.75,12.54,38.14,38.0,1.48,1.67,35.77,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,MEDANTA,1087.93,1486.00,11.34,H-MC,13.94,158966.0,32766.0,13417.0,2.02,25.96,8.44,36.59,108.0,2.44,1.17,41.73,XY24,NTT,HEALTHCARE
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.40,132678.0,7381.0,51625.0,2.21,5.89,38.91,47.09,141.0,0.14,0.98,66.06,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,10.70,H-MC,6.25,191130.0,26460.0,41513.0,-0.23,16.07,21.72,41.28,89.0,0.64,1.41,25.33,X40N,ATH,AC
91,VOLTAS,1278.28,1918.49,-1.92,H-MC,7.60,205725.0,13983.0,82043.0,1.65,7.29,39.88,50.08,99.0,0.17,1.51,14.59,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,9.41,177580.0,26667.0,30633.0,1.80,17.67,17.25,37.97,93.0,0.87,1.31,47.45,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,9.41,177580.0,26667.0,30633.0,1.80,17.67,17.25,37.97,93.0,0.87,1.31,47.45,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.97,H-SC,11.92,139708.0,16372.0,117397.0,2.45,13.27,84.03,108.45,119.0,0.14,1.03,38.59,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,11.32,H-SC,11.40,132678.0,7381.0,51625.0,2.21,5.89,38.91,47.09,141.0,0.14,0.98,66.06,MH,ATH,POWER
92,WHIRLPOOL,1219.98,2270.00,-42.81,M-SC,4.76,96622.0,5124.0,73626.0,-0.89,5.60,76.20,86.07,223.0,0.07,0.71,39.77,XR,NTT,DURABLES
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101310.0,806.0,29836.0,1.50,0.80,29.45,30.49,79.0,0.03,0.75,33.47,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.39,73997.0,-39552.0,79606.0,1.23,-34.83,107.58,35.27,268.0,-0.50,0.54,81.11,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.49,H-SC,15.49,90730.0,-10233.0,104766.0,1.16,-10.14,115.47,93.63,148.0,-0.10,0.67,35.14,SR,ATH,CHEMICALS
89,VALIANTORG,512.64,838.00,-322.08,H-SC,16.00,94380.0,-38906.0,123496.0,-1.94,-29.19,130.85,63.47,139.0,-0.32,0.69,58.20,XR,NTT,CHEMICALS
55,MASFIN,326.60,399.50,-17.80,H-SC,12.71,93465.0,-4515.0,26385.0,1.55,-4.61,28.23,22.32,152.0,-0.17,0.69,35.56,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-10.13,H-LC,7.11,96554.0,-1191.0,38583.0,-1.15,-1.22,39.96,38.26,35.0,-0.03,0.71,27.10,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.00,-20.67,H-SC,11.77,237143.0,44123.0,3913.0,1.71,22.86,1.65,24.89,137.0,11.28,1.74,51.54,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,9.41,177580.0,26667.0,30633.0,1.80,17.67,17.25,37.97,93.0,0.87,1.31,47.45,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-9.31,H-LC,7.16,226454.0,41973.0,28397.0,1.39,22.75,12.54,38.14,38.0,1.48,1.67,35.77,X40N,NTT,FMCG
5,ASIANPAINT,2961.56,4250.00,-10.32,H-LC,4.19,174896.0,-26490.0,114102.0,-0.60,-13.15,65.24,43.51,27.0,-0.23,1.29,21.75,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.86,H-LC,6.59,172172.0,15995.0,10106.0,0.17,10.24,5.87,16.71,22.0,1.58,1.27,29.47,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,40.97
2,50,70.49


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.63
LC    35.39
MC    26.00
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.14
X40N     15.39
X40      14.11
XY25     11.49
XR        8.83
AR        8.32
OX40N     7.37
X5K       2.20
MH        1.73
X200      1.34
SR        1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.24
H-SC    24.09
H-MC    23.11
M-SC    13.10
M-LC     7.10
M-MC     2.56
L-SC     1.44
L-LC     1.05
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.05,-1.43,32.48
IT,11.54,-18.96,78.37
FINANCE,8.02,-19.68,63.96
BANKS,7.43,-12.86,60.78
MISC,5.74,-17.74,85.23
PAINTS,5.59,-15.11,37.79
HEALTHCARE,5.22,7.43,15.95
ELECTRICAL,4.67,-10.67,58.59
AUTO,4.56,-9.21,56.72


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2810803.0,23
XR,1092929.0,13
AR,1078273.0,10
X40,819822.0,11
X40N,518149.0,12
XY25,484773.0,8
OX40N,457562.0,10
SR,194446.0,2
X5K,138993.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2745970.0,28
M-SC,1463179.0,17
H-MC,1238926.0,18
H-LC,1200578.0,19
M-LC,411991.0,5
M-MC,329094.0,2
L-SC,264374.0,3
L-MC,57906.0,1
L-LC,43449.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,931465.0,6
M-SC,XY24,863571.0,7
H-SC,AR,657824.0,5
H-LC,X40,530897.0,6
H-SC,XR,516907.0,6
H-MC,XY24,483254.0,5
M-MC,XY24,329094.0,2
H-LC,AR,232700.0,2
H-MC,X40,208498.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
